<a href="https://colab.research.google.com/github/earltreloar/logosfield-cddr-analysis/blob/main/Final_SPIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# sim_mech1_alignment_runner_v2.ipynb — one-cell runner
# Reproduces the tuned-v2 simulation, prints results, and bundles outputs.

import numpy as np, pandas as pd, os, json, math, zipfile
import matplotlib.pyplot as plt

# -------------------- Configuration --------------------
N   = 100_000
MC  = 600
SEED= 22
BINS = [(0.0, 0.5, 0.64),
        (0.5, 1.0, 0.68),
        (1.0, 2.0, 0.72)]

OUTDIR = "/content/sim_mech1_alignment_tuned_v2"
ZIP    = "/content/sim_mech1_alignment_tuned_v2_bundle.zip"
os.makedirs(OUTDIR, exist_ok=True)

# -------------------- Helpers --------------------
rng = np.random.default_rng(SEED)

def ra_dec_to_unit(ra_deg, dec_deg):
    ra  = np.deg2rad(ra_deg)
    dec = np.deg2rad(dec_deg)
    x = np.cos(dec)*np.cos(ra)
    y = np.cos(dec)*np.sin(ra)
    z = np.sin(dec)
    return np.vstack([x,y,z]).T

def assign_spins(expected_sign, p_align, rng):
    flips = rng.random(len(expected_sign)) > p_align
    spins = expected_sign.copy()
    spins[flips] *= -1
    return spins

def log10_bayes_factor_binomial(n, k, p0):
    if n == 0 or p0<=0 or p0>=1: return np.nan
    log10_m1 = -math.log10(n + 1.0)
    lnC = math.lgamma(n+1) - math.lgamma(k+1) - math.lgamma(n-k+1)
    log10C = lnC / math.log(10.0)
    log10_m0 = log10C + k*math.log10(p0) + (n-k)*math.log10(1.0 - p0)
    return log10_m1 - log10_m0

# -------------------- Synthetic catalog --------------------
ra  = rng.uniform(0, 360, size=N)
u   = rng.uniform(-1, 1, size=N)
dec = np.rad2deg(np.arcsin(u))
z   = np.clip(np.random.default_rng(5).gamma(shape=2.0, scale=0.5, size=N),0,2)

L0  = np.array([0.4, -0.3, 0.865]); L0 /= np.linalg.norm(L0)
xyz = ra_dec_to_unit(ra, dec)
Q   = np.array([[0.2,0,0],[0,-0.1,0],[0,0,0.05]])
Ln  = L0 + 0.3*(xyz @ Q.T); Ln /= np.linalg.norm(Ln, axis=1, keepdims=True)
dots = np.sum(xyz*Ln, axis=1)
expected_sign = np.where(dots>=0, 1, -1)

p_align = np.zeros(N)
for zmin, zmax, p in BINS:
    mask = (z>=zmin)&(z<zmax); p_align[mask] = p
spins = assign_spins(expected_sign, p_align, rng)

# -------------------- Stats per bin --------------------
rows = []
for zmin, zmax, p in BINS:
    idx = np.where((z>=zmin)&(z<zmax))[0]
    n = len(idx); k = (spins[idx]==expected_sign[idx]).sum(); frac = k/n
    hits = [ (np.random.permutation(spins[idx])==expected_sign[idx]).sum()/n for _ in range(MC) ]
    p0_mean, p0_sd = np.mean(hits), np.std(hits, ddof=1)
    log10BF = log10_bayes_factor_binomial(n, k, p0_mean)
    rows.append({"zmin":zmin,"zmax":zmax,"target_p_align":p,
                 "n":n,"k_aligned":int(k),
                 "observed_fraction":frac,"null_mean":p0_mean,
                 "null_sd":p0_sd,"log10BF":log10BF})
bin_table = pd.DataFrame(rows)

# Pooled
n_tot = len(spins); k_tot = (spins==expected_sign).sum(); frac_tot = k_tot/n_tot
hits_tot = [ (np.random.permutation(spins)==expected_sign).sum()/n_tot for _ in range(MC) ]
p0_mean_tot, p0_sd_tot = np.mean(hits_tot), np.std(hits_tot, ddof=1)
log10BF_tot = log10_bayes_factor_binomial(n_tot, k_tot, p0_mean_tot)
pooled = {"k":int(k_tot),"n":int(n_tot),"p_obs":float(frac_tot),
          "p0_mean":p0_mean_tot,"p0_sd":p0_sd_tot,"log10BF":log10BF_tot}

# -------------------- Save outputs --------------------
bin_csv   = os.path.join(OUTDIR,"alignment_by_zbin_tuned_v2.csv")
pooled_js = os.path.join(OUTDIR,"alignment_pooled_tuned_v2.json")
summary   = os.path.join(OUTDIR,"alignment_summary_tuned_v2.txt")
bin_table.to_csv(bin_csv,index=False)
with open(pooled_js,"w") as f: json.dump(pooled,f,indent=2)

with open(summary,"w") as f:
    f.write("Mechanism #1 — Simulated Alignment (tuned v2)\n")
    f.write("="*60+"\n\n")
    f.write(bin_table.to_string(index=False)+"\n\n")
    f.write("Pooled results:\n")
    for k,v in pooled.items():
        f.write(f"{k}: {v}\n")

# -------------------- Print results to screen --------------------
print("Per-bin results:")
print(bin_table)
print("\nPooled:")
print(pooled)

# -------------------- Plots --------------------
plt.figure()
x = np.arange(len(BINS))
plt.errorbar(x, bin_table["null_mean"], yerr=bin_table["null_sd"], fmt='o', label="Null mean ± sd")
plt.plot(x, bin_table["observed_fraction"], 's', label="Observed")
plt.xticks(x, [f"{b[0]}–{b[1]}" for b in BINS])
plt.xlabel("Redshift bin (z)"); plt.ylabel("Alignment fraction")
plt.title("Spin alignment vs null by redshift bin (tuned v2)"); plt.legend()
plot1 = os.path.join(OUTDIR,"plot_alignment_by_bin_tuned_v2.png")
plt.savefig(plot1,dpi=140,bbox_inches="tight"); plt.close()

plt.figure()
plt.hist(hits_tot, bins=30); plt.axvline(pooled["p_obs"], linestyle="--",color="red")
plt.xlabel("Alignment fraction under null (pooled)"); plt.ylabel("Frequency")
plt.title("Pooled null vs observed (tuned v2)")
plot2 = os.path.join(OUTDIR,"plot_pooled_null_hist_tuned_v2.png")
plt.savefig(plot2,dpi=140,bbox_inches="tight"); plt.close()

# -------------------- Bundle everything --------------------
with zipfile.ZipFile(ZIP,"w",zipfile.ZIP_DEFLATED) as zf:
    for f in [bin_csv, pooled_js, summary, plot1, plot2]:
        zf.write(f, arcname=os.path.basename(f))

print("\nAll outputs zipped at:", ZIP)


Per-bin results:
   zmin  zmax  target_p_align      n  k_aligned  observed_fraction  null_mean  \
0   0.0   0.5            0.64  26484      17076           0.644767   0.500065   
1   0.5   1.0            0.68  32973      22494           0.682195   0.500095   
2   1.0   2.0            0.72  31198      22467           0.720142   0.499943   

    null_sd      log10BF  
0  0.003052   486.504054  
1  0.002644   969.756228  
2  0.002860  1357.845457  

Pooled:
{'k': 62037, 'n': 100000, 'p_obs': 0.62037, 'p0_mean': np.float64(0.4998746), 'p0_sd': np.float64(0.0015748003578056832), 'log10BF': np.float64(1271.1487293693785)}

All outputs zipped at: /content/sim_mech1_alignment_tuned_v2_bundle.zip
